# Bulk Load Json

In [1]:
%load_ext autoreload
%autoreload 2
from cord import *
from cord.core import *
from cord.text import preprocess

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dwight\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# JSON Catalog

In [19]:
for subset in [COMM_USE_SUBSET, BIORXIV_MEDRXIV, NONCOMM_USE_SUBSET, CUSTOM_LICENSE]:
    json_catalog = JCatalog.load(subset)
    %time json_catalog.nlp()
    json_catalog.save(subset)

Creating index tokens
Wall time: 3min 51s
Saving to data\JsonCatalog_comm_use_subset.pickle


Creating index tokens
Wall time: 13.1 s
Saving to data\JsonCatalog_biorxiv_medrxiv.pickle


Creating index tokens
Wall time: 46.2 s
Saving to data\JsonCatalog_noncomm_use_subset.pickle


Creating index tokens
Wall time: 7min 22s
Saving to data\JsonCatalog_custom_license.pickle


In [20]:
#for catalog in JSON_CATALOGS:
#    json_catalog = JCatalog.from_pickle(catalog)
#json_catalog = JCatalog.load(BIORXIV_MEDRXIV)
#json_catalog.nlp()
#json_catalog.save(BIORXIV_MEDRXIV)
json_catalog = JCatalog.from_pickle(BIORXIV_MEDRXIV)

Loading catalog biorxiv_medrxiv


## Research Papers

In [64]:
research_papers = ResearchPapers.from_data_dir()


Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Fixing dates that are a list e.g. "['2020-02-05', '2020-02']"
Fixing dates with the seasons e.g. "2014 Autumn"
Fixing dates like "2016 Nov 9 Jan-Feb"
Fixing dates like "2012 Jan-Mar"
Converting dates like "2020 Apr 13"
Converting Dates like "2020 Apr"
Converting dates like "2020"
Converting Dates like "2020-01-21"
Indexing research papers
Finished Indexing in 80.0 seconds


In [6]:
json_catalog.papers.index_tokens

0      [vp3, vp0, processed, vp2, vp4, virus, assembl...
1      [introduction, 2019-ncov, epidemic, spread, ac...
2      [introduction, infectious, bronchitis, ib, cau...
3      [introduction, nipah, infectious, negative-sen...
4      [introduction, december, cluster, patients, pn...
                             ...                        
880    [regulates, innate, immune, response, clinical...
881    [introduction, excess, production, oligomeriza...
882    [cc-by-nc-nd, international, license, made, av...
883    [introduction, infected, eg, wearing, mask, wa...
884    [virus-based, host-based, treatment, options, ...
Name: index_tokens, Length: 885, dtype: object

In [31]:
metadata = research_papers.metadata

In [8]:
metadata = research_papers.metadata
multiple_papers = metadata.sha.fillna('').str.contains(';')
no_papers = metadata.sha.isnull()
one_paper = ~(no_papers) & ~(multiple_papers)
metadata = metadata.loc[one_paper]

In [33]:
metadata.index_tokens

39181    [next-generation, sequencing, increasingly, us...
39182    [emerging, disease, one, infectious, epidemic,...
39183    [germline, variation, immunoglobulin, gene, ig...
39184    [deep, sequencing, clinical, samples, establis...
39185    [backgrounddeveloping, methods, reconstruct, t...
                               ...                        
40226    [governments, around, world, must, rapidly, mo...
40227    [2019-novel-coronavirus, covid-19, affected, c...
40228    [object, meteorological, parameters, important...
40229    [background, monitoring, time-varying, reprodu...
40230    [report, temporal, patterns, viral, shedding, ...
Name: index_tokens, Length: 1020, dtype: object

In [18]:
metadata_tokens = metadata.merge(json_catalog.papers, how='left', on='sha').index_tokens

In [35]:
json_catalog.papers.index_tokens

0      [vp3, vp0, processed, vp2, vp4, virus, assembl...
1      [introduction, 2019-ncov, epidemic, spread, ac...
2      [introduction, infectious, bronchitis, ib, cau...
3      [introduction, nipah, infectious, negative-sen...
4      [introduction, december, cluster, patients, pn...
                             ...                        
880    [regulates, innate, immune, response, clinical...
881    [introduction, excess, production, oligomeriza...
882    [cc-by-nc-nd, international, license, made, av...
883    [introduction, infected, eg, wearing, mask, wa...
884    [virus-based, host-based, treatment, options, ...
Name: index_tokens, Length: 885, dtype: object

In [43]:
from IPython.display import display
def set_index_tokens(metadata, json_papers):
    multiple_papers = metadata.sha.fillna('').str.contains(';')
    one_paper = ~( metadata.sha.isnull()) & ~(multiple_papers)
    old_tokens = metadata.index_tokens.copy()
    sha_index_tokens = ['sha', 'index_tokens']
    metadata = metadata[['sha']].merge(json_papers, how='left', on='sha').fillna(old_tokens).index_tokens
    display(mrg)
    
set_index_tokens(metadata, json_catalog.papers)

In [22]:
from IPython.display import display

In [62]:
research_papers = ResearchPapers.from_pickle()
metadata = research_papers.metadata

def lookup_tokens(shas, token_map):
    if not isinstance(shas, str): return []
    for sha in shas.split(';'):
        tokens = token_map.get(sha.strip())
        if tokens:
            return tokens
        
for catalog in JSON_CATALOGS:
    # Subset the metadata
    catalog_idx = research_papers.metadata.full_text_file == catalog
    metadata_papers = metadata.loc[catalog_idx, ['sha']].copy().reset_index()
    
    # Load the json catalog
    json_catalog = JCatalog.from_pickle(catalog)
    json_papers = json_catalog.papers[['sha', 'index_tokens']].reset_index(drop=True)
    
    # Set the index tokens from the json_papers to the metadata
    sha_tokens = metadata_papers.merge(json_papers, how='left', on='sha').set_index('index')
    
    # Handle records with multiple shas
    has_multiple = (sha_tokens.sha.fillna('').str.contains(';'))
    token_map = json_papers[['sha', 'index_tokens']].set_index('sha').to_dict()['index_tokens']
    sha_tokens.loc[has_multiple, 'index_tokens']  \
            = sha_tokens.loc[has_multiple, 'sha'].apply(lambda sha: lookup_tokens(sha, token_map) )
    
    metadata.loc[catalog_idx, 'index_tokens'] = sha_tokens.index_tokens

Loading catalog comm_use_subset
Loading catalog biorxiv_medrxiv
Loading catalog noncomm_use_subset
Loading catalog custom_license


In [63]:
research_papers.get_papers(CUSTOM_LICENSE)

Indexing research papers


TypeError: object of type 'float' has no len()

In [10]:
token_map = cat.papers[['sha', 'index_tokens']].set_index('sha').to_dict()['index_tokens']

In [13]:
token_map['000b7d1517ceebb34e1e3e817695b6de03e2fa78'];
research_papers = ResearchPapers.from_pickle()
research_papers = research_papers.get_papers(COMM_USE_SUBSET)
metadata = research_papers.metadata


Indexing research papers
Finished Indexing in 12.0 seconds


In [20]:
has_multiple = metadata.sha.fillna('').str.contains(';')

def lookup_tokens(shas):
    for sha in shas.split(';'):
        tokens = token_map.get(sha.strip())
        if tokens:
            return tokens
        
tokens = metadata.loc[has_multiple, ['sha']].sha.apply(lookup_tokens)
tokens

40237    [temperature, dependence, yeast, invertase-fpc...
40242    [introduction, influenza, virus, notable, publ...
40259    [supplementary, note, -host-pathogen, interact...
40287    [서론, introduction, 2012년, 감사의, acknowledgement...
40291    [supplementary, figure, comparison, mers-cov, ...
                               ...                        
43919    [date, full-length, genome, sequence, caastv, ...
43921    [introduction, use, extracorporeal, membrane, ...
43924    [cell-specific, marker, gene, analysis, first,...
43928    [background, pneumonia, leading, cause, morbid...
43930    [viral, etiology, hospitalized, acute, lower, ...
Name: sha, Length: 375, dtype: object

In [21]:
tokens.isnull().any()

False

In [40]:
pd.options.display.max_colwidth = 120